In [1]:
import json
import os
import pandas as pd
from tqdm import tqdm
requests_dir = '../data/input/requests'
response_dir = '../data/input/responses'


import sys
sys.path.append('../utils')
import helpers as h

## Combining all routes in a single df.
We'll output a single dataframe that has all route requests read and parsed into a single df.

In [2]:
def read_request(route_id, ymd, idx_file):
    """
    Gepaste docstring in het goede formaat.
    Als je dit in het goede formaat doet kan je meteen documentatie maken
    https://realpython.com/python-project-documentation-with-mkdocs/

    Functie van Tim
    """
    
    folder_requests = os.path.join('..', 'data', 'input', 'requests', f"{route_id}-{ymd}")
    if not os.path.exists(folder_requests):
        return None

    file_request = os.listdir(folder_requests)[idx_file]
    file_path_request = os.path.join(folder_requests, file_request)
    print(file_path_request)
    with open(file_path_request, 'r') as f:
        request = json.load(f)
    print('OK')
    rows = list()
    for task in request['tasks']:
        # TODO: Dit kan efficienter met een specifieke methode (pd.explode, json to dataframe zaken)
        if task['id'] == 'E1':
            continue ## TODO: Also retain this task!
        row = {'id' : int(task['id']),
                    'lat' : task['address']['latitude'],
                    'long' : task['address']['longitude'],
                    'start_time' : task['timeWindow']['from'],
                    'end_time' : task['timeWindow']['till']}
        rows.append(row)

    return pd.DataFrame(rows).sort_values(by = 'id')
def get_route_dataframe(route_id, ymd, idx_file): 
    """
        Simple one linter that uses read_reques and sort_request to
        get the route dataframe from the JSON file, can be used as
        input for other functions. 
    """
    df_request = read_request(route_id, ymd, idx_file)
    #df_request = sort_request(df_request, route_id, ymd, idx_file)
    return df_request




In [3]:

def json_request_to_df(path): 
    """
        variation on Tim's function 'read_request', we'll not let a funciton decide
        what index file (last integer iin the filename) to read. in stead we do this
        in the df itself applying filters where needed. Gives us more control in case
        a json response for a hardcoded idx in read_requst is invalid. 

        CHOICES MADE: stop order is determined by JSON order!!!!!
        //TODO ==> Should be written in docfile.

        ARGUMENTS: 
            path = str = Fully qualified path to a json file

        RETURNS: 
            pandas dataframe
    """
    with open(path, 'r') as f:
        request = json.load(f)
    rows = list()
    #meta: 
    file_uuid = request['id']
    configurationName = request['configurationName']
    stop_number = 1
    for task in request['tasks']:
        # TODO: Dit kan efficienter met een specifieke methode (pd.explode, json to dataframe zaken)
        if task['id'] == 'E1':
            continue ## TODO: Also retain this task!
        row = { 
                'file_uuid' : file_uuid,
                'configurationName' : configurationName,
                'id' : int(task['id']),
                'lat' : task['address']['latitude'],
                'long' : task['address']['longitude'],
                'start_time' : task['timeWindow']['from'],
                'end_time' : task['timeWindow']['till'], 
                'stop_number': stop_number
                }
        stop_number += 1
        rows.append(row)
    #we assume the server JSON response is ordered, so we need the stop_number to increment to build up routes.
    return pd.DataFrame(rows).sort_values(by = 'stop_number')

#reading requests: 
internal_id = 1     #using internal_id we can quikly comminicate with each other what precise file we want to look at for debugging/app-feature design. 
all_content = []    # the final dataframe (build once all data is read)
for dir in tqdm(os.listdir(requests_dir)):
    if dir.endswith(".txt"):
        continue
    route_id = dir.split('-')[0]
    route_date = h.routedatestring_to_date(dir.split('-')[1])
    contents = os.listdir(os.path.join(requests_dir, dir))
    for file in contents:
        if file.endswith('.json'): 
            fn = file.rstrip('.json').split('-')
            idblock_1 = fn[2]
            idblock_2 = fn[3]
            idblock_3 = fn[4]
            fq_path = os.path.join(requests_dir, dir, file)
            content_of_file = json_request_to_df(fq_path)
            #enabling us to identify rows belong to a specific file on the drive. 
            content_of_file['dir'] = dir
            content_of_file['route_id'] = route_id
            content_of_file['route_date'] = route_date
            content_of_file['idblock_1'] = idblock_1
            content_of_file['idblock_2'] = idblock_2
            content_of_file['idblock_3'] = idblock_3
            content_of_file['internal_id'] = internal_id
            internal_id+=1
            all_content.append(content_of_file)


100%|██████████| 3726/3726 [02:08<00:00, 29.05it/s]


In [4]:
df_requests = pd.concat(all_content)

In [5]:
os.makedirs('../data/intermediate', exist_ok=True)
df_requests.to_csv('../data/intermediate/requests.csv', index=False)


## Reading excel data
UUID's can be matched to recover timestamp information - required for EDA phase and as a way to dedouble points. 

In [6]:
excel_df = pd.read_excel('../data/input/ModifiedQueryRows.xlsx')

In [7]:
excel_df.rename(columns={' Time': 'Time'}, inplace=True)

In [8]:
excel_df.to_csv('../data/intermediate/serverlog.csv')

## Connecting serverlog to JSON files: 
We need to connect the two logs to reliably determine: 
- the order of requests
- initial route suggestion vs route driven

In [9]:
#In the explanation of the data we were basically told to ignore all requests in the PM-range of the day. Do that here: 
server_df = excel_df[~excel_df['Time'].str.contains('PM')]
#with the UUID of the request we can no allign the request time for a route_id on a day. This allows us to
#recover the final driven route and have the scatterplot dedoubled 
server_df

,Date,Time,OptimizationRequestId,RouteId,TriggerType,ConfigurationName,NumberOfTasks,NumberOfTasksInInputPlan
20,5/30/2022,10:39:12.287 AM,094aeb3f-4e9f-48e4-a14c-97adce541cf0,0512_E46,TimeCalculation,EstimateTime,38,38
21,5/30/2022,10:39:44.836 AM,2509a3a1-5e40-4357-bb87-9d73426cf40c,0512_512,TimeCalculation,EstimateTime,58,58
22,5/30/2022,10:40:20.568 AM,5a07fa18-c340-4d61-963e-56f1812a6d60,0515_I06,FullOptimization,CreateSequence,14,0
23,5/30/2022,10:40:20.850 AM,ae3dd9c1-c7be-4536-9b59-688a5a9837dd,0521_698,TimeCalculation,EstimateTime,154,154
24,5/30/2022,10:40:29.748 AM,c35395ec-b54c-4dc9-a1f1-80a730c079f7,0521_860,TimeCalculation,EstimateTime,135,135
...,...,...,...,...,...,...,...,...
134299,6/9/2022,9:56:42.831 AM,1b106804-f0e8-4671-acc5-712a019fc422,0511_163,TimeCalculation,EstimateTime,63,63
134300,6/9/2022,9:57:05.806 AM,e623ae2e-854a-49a5-a861-ced0bed1fce3,0521_375,TimeCalculation,EstimateTime,72,72
134301,6/9/2022,9:57:47.814 AM,099898c7-cd77-4b93-bc2a-df5e74a29201,0511_157,FullOptimization,CreateSequence,110,0
134302,6/9/2022,9:58:47.785 AM,276c2223-c067-46b0-8a8d-17f8d7b6abf4,0521_887,TimeCalculation,EstimateTime,157,157


In [10]:
#remove rows where RouteId == shiftId
server_df = server_df.loc[server_df['RouteId'] != 'shiftId']

In [11]:
server_df

,Date,Time,OptimizationRequestId,RouteId,TriggerType,ConfigurationName,NumberOfTasks,NumberOfTasksInInputPlan
20,5/30/2022,10:39:12.287 AM,094aeb3f-4e9f-48e4-a14c-97adce541cf0,0512_E46,TimeCalculation,EstimateTime,38,38
21,5/30/2022,10:39:44.836 AM,2509a3a1-5e40-4357-bb87-9d73426cf40c,0512_512,TimeCalculation,EstimateTime,58,58
22,5/30/2022,10:40:20.568 AM,5a07fa18-c340-4d61-963e-56f1812a6d60,0515_I06,FullOptimization,CreateSequence,14,0
23,5/30/2022,10:40:20.850 AM,ae3dd9c1-c7be-4536-9b59-688a5a9837dd,0521_698,TimeCalculation,EstimateTime,154,154
24,5/30/2022,10:40:29.748 AM,c35395ec-b54c-4dc9-a1f1-80a730c079f7,0521_860,TimeCalculation,EstimateTime,135,135
...,...,...,...,...,...,...,...,...
134299,6/9/2022,9:56:42.831 AM,1b106804-f0e8-4671-acc5-712a019fc422,0511_163,TimeCalculation,EstimateTime,63,63
134300,6/9/2022,9:57:05.806 AM,e623ae2e-854a-49a5-a861-ced0bed1fce3,0521_375,TimeCalculation,EstimateTime,72,72
134301,6/9/2022,9:57:47.814 AM,099898c7-cd77-4b93-bc2a-df5e74a29201,0511_157,FullOptimization,CreateSequence,110,0
134302,6/9/2022,9:58:47.785 AM,276c2223-c067-46b0-8a8d-17f8d7b6abf4,0521_887,TimeCalculation,EstimateTime,157,157


In [12]:
#Date AND Time Conversion
server_df.loc[:, 'Time'] = pd.to_datetime(server_df['Time'], format=' %I:%M:%S.%f %p') #weird space in string - hack
server_df.loc[:, 'Date'] = pd.to_datetime(server_df['Date'], format='%m/%d/%Y')

In [13]:
ids = []
is_max = []
for _, df in server_df.groupby(['Date', 'RouteId']):
    df = df.sort_values(by='Time')
    id_range = range(1, len(df) + 1)
    ids.extend(id_range) 
    is_max.extend([False] * (len(df) - 1) + [True])
server_df.loc[:, 'request_day_route_order'] = ids
server_df.loc[:, 'request_day_route_is_last'] = is_max

C:\Users\Frede\AppData\Local\Temp\ipykernel_22104\3012880848.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  server_df.loc[:, 'request_day_route_order'] = ids
C:\Users\Frede\AppData\Local\Temp\ipykernel_22104\3012880848.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  server_df.loc[:, 'request_day_route_is_last'] = is_max


In [14]:
server_df.isna().sum()#good

Date                         0
Time                         0
OptimizationRequestId        0
RouteId                      0
TriggerType                  0
ConfigurationName            0
NumberOfTasks                0
NumberOfTasksInInputPlan     0
request_day_route_order      0
request_day_route_is_last    0
dtype: int64

In [15]:
#apply the request_day_route_is_last to requests_df and save into a smaller df (optimized)
merged_df = pd.merge(df_requests, server_df, left_on='file_uuid', right_on='OptimizationRequestId', how='left')
request_df_driven = merged_df.query('request_day_route_is_last==True')

In [16]:
merged_df.isna().sum()#not good: but we know what this is; these are the PM requests. now we can drop them too from the initial file!

file_uuid                         0
configurationName                 0
id                                0
lat                               0
long                              0
start_time                        0
end_time                          0
stop_number                       0
dir                               0
route_id                          0
route_date                        0
idblock_1                         0
idblock_2                         0
idblock_3                         0
internal_id                       0
Date                         455737
Time                         455737
OptimizationRequestId        455737
RouteId                      455737
TriggerType                  455737
ConfigurationName            455737
NumberOfTasks                455737
NumberOfTasksInInputPlan     455737
request_day_route_order      455737
request_day_route_is_last    455737
dtype: int64

In [17]:
merged_df = merged_df.dropna()

In [18]:
print("SHAPE: ", merged_df.shape)
print("NANS: \n", merged_df.isna().sum())
#good!

SHAPE:  (2120376, 25)
NANS: 
 file_uuid                    0
configurationName            0
id                           0
lat                          0
long                         0
start_time                   0
end_time                     0
stop_number                  0
dir                          0
route_id                     0
route_date                   0
idblock_1                    0
idblock_2                    0
idblock_3                    0
internal_id                  0
Date                         0
Time                         0
OptimizationRequestId        0
RouteId                      0
TriggerType                  0
ConfigurationName            0
NumberOfTasks                0
NumberOfTasksInInputPlan     0
request_day_route_order      0
request_day_route_is_last    0
dtype: int64


In [19]:
merged_df.to_csv("../data/intermediate/clean_data.csv", index=False)